In [ ]:
!pip install pulp

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

In [10]:
# Lista składników
Ingredients = ["CHICKEN", "BEEF", "SARDINES", "MUTTON", "RICE", "WHEAT_BRAN", "GEL"]

# Koszty
costs = {
    "CHICKEN": 0.013,
    "BEEF": 0.008,
    "SARDINES": 0.00008,
    "MUTTON": 0.010,
    "RICE": 0.002,
    "WHEAT_BRAN": 0.005,
    "GEL": 0.001,
}

# Zawartość białka (w procentach)
proteinPercent = {
    "CHICKEN": 0.100,
    "BEEF": 0.200,
    "SARDINES": 0.050,
    "MUTTON": 0.150,
    "RICE": 0.000,
    "WHEAT_BRAN": 0.040,
    "GEL": 0.000,
}

# Zawartość tłuszczu (w procentach)
fatPercent = {
    "CHICKEN": 0.080,
    "BEEF": 0.100,
    "SARDINES": 0.150,
    "MUTTON": 0.110,
    "RICE": 0.010,
    "WHEAT_BRAN": 0.010,
    "GEL": 0.000,
}

# Zawartość błonnika (w procentach)
fibrePercent = {
    "CHICKEN": 0.001,
    "BEEF": 0.005,
    "SARDINES": 0.000,
    "MUTTON": 0.003,
    "RICE": 0.100,
    "WHEAT_BRAN": 0.150,
    "GEL": 0.000,
}

# Zawartość soli (w procentach)
saltPercent = {
    "CHICKEN": 0.002,
    "BEEF": 0.005,
    "SARDINES": 0.010,
    "MUTTON": 0.007,
    "RICE": 0.002,
    "WHEAT_BRAN": 0.008,
    "GEL": 0.000,
}

# Zawartość witaminy B (w jednostkach na porcję)
vitaminBPercent = {
    "CHICKEN": 0.3,
    "BEEF": 2.6,
    "SARDINES": 8.9,
    "MUTTON": 2.6,
    "RICE": 0.0,
    "WHEAT_BRAN": 0.0,
    "GEL": 0.0,
}


In [11]:
# Problem optymalizacyjny
cat_food_problem = LpProblem("Cat_Food_Optimization", LpMinimize)

# Zmienne decyzyjne (udział procentowy każdego składnika)
ingredient_vars = {i: LpVariable(f"{i}_percent", 1, 100) for i in ingredients}  # Min 1% udziału

# Minimalizacja kosztu składników
cat_food_problem += lpSum([ingredients[i]["cost"] * ingredient_vars[i] for i in ingredients]), "Total Cost"

# Ograniczenia składników odżywczych
cat_food_problem += lpSum([ingredient_vars[i] for i in ingredients]) == 100, "TotalPercentage"
cat_food_problem += lpSum([ingredients[i]["protein"] * ingredient_vars[i] for i in ingredients]) >= 8.0, "ProteinRequirement"
cat_food_problem += lpSum([ingredients[i]["fat"] * ingredient_vars[i] for i in ingredients]) >= 6.0, "FatRequirement"
cat_food_problem += lpSum([ingredients[i]["fibre"] * ingredient_vars[i] for i in ingredients]) <= 2.0, "FibreRequirement"
cat_food_problem += lpSum([ingredients[i]["salt"] * ingredient_vars[i] for i in ingredients]) <= 0.4, "SaltRequirement"

# Minimalna zawartość witaminy B
cat_food_problem += lpSum([ingredients[i]["vitaminB"] * ingredient_vars[i] for i in ingredients]) >= 0.8, "VitaminBRequirement"



In [12]:
# Rozwiązanie problemu
cat_food_problem.solve()

# Wyświetlenie wyników
print("Status:", LpStatus[cat_food_problem.status])

# Zapisanie wyniku w czytelnej formie
total_cost = round(value(cat_food_problem.objective), 2)
composition = {v.name.replace("_percent", ""): round(v.varValue, 2) for v in cat_food_problem.variables()}

# Wydruk podsumowania
print(f"Końcowy produkt będzie kosztować {total_cost} zł i będzie miał w składzie:")
for ingredient, percent in composition.items():
    print(f"- {ingredient}: {percent}%")

Status: Optimal
Końcowy produkt będzie kosztować 0.34 zł i będzie miał w składzie:
- beef: 33.17%
- chicken: 1.0%
- gel: 41.31%
- mutton: 1.0%
- rice: 1.0%
- sardines: 21.51%
- wheat_bran: 1.0%
